In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DebitCredit").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/26 22:59:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/26 22:59:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [32]:
df = spark.read.csv("DataSets/debit_credit.csv",header=True)
df.show()

+---+-----+---------------+------+
|Cid|CName|TransactionType|Amount|
+---+-----+---------------+------+
|101|  Nik|          debit|    10|
|101|  Nik|         credit|   100|
|101|  Nik|          debit|    50|
|102| Raki|         credit|  1000|
|102| Raki|         credit|   100|
|102| Raki|          debit|    50|
|103|Nibbi|         credit|  1000|
|104|Nibba|          debit|  1000|
|104|Nibba|         credit|   400|
|104|Nibba|          debit|   400|
|103|Nibbi|         credit|   400|
+---+-----+---------------+------+



In [33]:
from pyspark.sql.functions import when,col,sum

In [34]:
df1 = df.withColumn("Amount_cr_db",when(col("TransactionType")=="debit",-1*col("Amount")).otherwise(col("Amount")))
df1.show()

+---+-----+---------------+------+------------+
|Cid|CName|TransactionType|Amount|Amount_cr_db|
+---+-----+---------------+------+------------+
|101|  Nik|          debit|    10|       -10.0|
|101|  Nik|         credit|   100|         100|
|101|  Nik|          debit|    50|       -50.0|
|102| Raki|         credit|  1000|        1000|
|102| Raki|         credit|   100|         100|
|102| Raki|          debit|    50|       -50.0|
|103|Nibbi|         credit|  1000|        1000|
|104|Nibba|          debit|  1000|     -1000.0|
|104|Nibba|         credit|   400|         400|
|104|Nibba|          debit|   400|      -400.0|
|103|Nibbi|         credit|   400|         400|
+---+-----+---------------+------+------------+



In [35]:
df1.groupBy(col("Cid")).agg(sum("Amount_cr_db").alias("Total Amount")).show()

+---+------------+
|Cid|Total Amount|
+---+------------+
|101|        40.0|
|104|     -1000.0|
|102|      1050.0|
|103|      1400.0|
+---+------------+



In [37]:
df2 = df.groupBy(["Cid","CName"]).pivot("TransactionType").agg(sum("Amount"))

df2.show()

+---+-----+------+------+
|Cid|CName|credit| debit|
+---+-----+------+------+
|101|  Nik| 100.0|  60.0|
|104|Nibba| 400.0|1400.0|
|102| Raki|1100.0|  50.0|
|103|Nibbi|1400.0|  null|
+---+-----+------+------+



In [41]:
df2 = df2.fillna(0)
df2.show()

+---+-----+------+------+
|Cid|CName|credit| debit|
+---+-----+------+------+
|101|  Nik| 100.0|  60.0|
|104|Nibba| 400.0|1400.0|
|102| Raki|1100.0|  50.0|
|103|Nibbi|1400.0|   0.0|
+---+-----+------+------+



In [42]:
df2.withColumn("Total Bal",col("credit")-col("debit")).show()

+---+-----+------+------+---------+
|Cid|CName|credit| debit|Total Bal|
+---+-----+------+------+---------+
|101|  Nik| 100.0|  60.0|     40.0|
|104|Nibba| 400.0|1400.0|  -1000.0|
|102| Raki|1100.0|  50.0|   1050.0|
|103|Nibbi|1400.0|   0.0|   1400.0|
+---+-----+------+------+---------+

